In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys
from tqdm import tqdm

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts")))

import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

In [3]:
from underwriting import data_quality_checks

In [4]:
def cogs_analysis(df_in:pd.DataFrame,df_sales:pd.DataFrame,freq: str) -> Tuple[pd.DataFrame,pd.DataFrame]:
    df_sales['per_unit'] = df_sales['tx_total_price'] / df_sales['tx_quantity_sold']
    # set frequency
    if freq == 'monthly':
        df_sales['date'] = df_sales['sales_datetime'].dt.strftime("%Y-%m")
    elif freq == 'weekly':
        df_sales['date'] = df_sales['sales_datetime'].dt.strftime("%Y-%W")
        df_sales['week'] = df_sales['sales_datetime'].dt.strftime("%W")
    # total # of trxns
    s_total_count = df_sales.groupby('date')['tx_total_price'].count()
    df_total_count = pd.Series(s_total_count).to_frame()
    df_total_count = df_total_count.reset_index()
    df_total_count.rename(columns={'tx_total_price':'total_count'}, inplace=True)
    # revenue
    s_revenue = df_sales.groupby('date')['tx_total_price'].sum()
    df_revenue = pd.Series(s_revenue).to_frame()
    df_revenue = df_revenue.reset_index()
    df_revenue.rename(columns={'tx_total_price': 'revenue'}, inplace=True)

    df_in['per_unit_incoming'] = df_in['shipper_wholesale_price'] / df_in['shipped_quantity']
    
    # per unit price by package id
    df_in_price = df_in[df_in['shipper_wholesale_price'].notnull()]
    average_incoming_package_id = df_in_price.groupby('package_id')['per_unit_incoming'].mean()
    df_avg_incoming_price = pd.Series(average_incoming_package_id).to_frame()
    df_avg_incoming_price = df_avg_incoming_price.reset_index()
    # per unit price by product name
    average_incoming_product = df_in_price.groupby('product_name')['per_unit_incoming'].mean()
    df_avg_product = pd.Series(average_incoming_product).to_frame()
    df_avg_product = df_avg_product.reset_index()
    df_avg_product.rename(columns={'per_unit_incoming':'per_unit_product'}, inplace=True)

    # merge with (cogs by package id)
    df_cogs_package_id = pd.merge(df_sales, df_avg_incoming_price, left_on='tx_package_id', right_on='package_id', how='left')
    df_cogs_package_id['total_incoming'] = df_cogs_package_id['per_unit_incoming'] * df_cogs_package_id['tx_quantity_sold']
    df_cogs_package_id.replace([numpy.inf], numpy.nan, inplace=True)
    df_cogs_package_id_notnull = df_cogs_package_id[df_cogs_package_id['total_incoming'].notnull()]

    # sum cogs by package id
    s_cogs = df_cogs_package_id_notnull.groupby('date')['total_incoming'].sum()
    df_cogs_id = pd.Series(s_cogs).to_frame()
    df_cogs_id = df_cogs_id.reset_index()
    # count # of trxn by package id
    s_cogs_count = df_cogs_package_id_notnull.groupby('date')['total_incoming'].count()
    df_cogs_count = pd.Series(s_cogs_count).to_frame()
    df_cogs_count = df_cogs_count.reset_index()
    df_cogs_count.rename(columns={'total_incoming':'count_incoming'}, inplace=True)
    
    # merge with (cogs by product name)
    df_cogs_average_product = pd.merge(df_cogs_package_id, df_avg_product, left_on='tx_product_name', right_on='product_name', how='left')
    df_cogs_average_product['total_product'] = df_cogs_average_product['tx_quantity_sold'] * df_cogs_average_product['per_unit_product']
    df_cogs_null = df_cogs_average_product[df_cogs_average_product['per_unit_incoming'].isnull()]
    df_cogs_product = df_cogs_null[df_cogs_null['per_unit_product'].notnull()]
    # sum cogs filldown by product name
    product_sum = df_cogs_product.groupby('date')['total_product'].sum()
    df_product_sum = pd.Series(product_sum).to_frame()
    df_product_sum = df_product_sum.reset_index()
    df_product_sum.rename(columns={'total_product':'product_sum'}, inplace=True)
    # count # of trxn filldown by product name
    product_count = df_cogs_product.groupby('date')['total_product'].count()
    df_product_count = pd.Series(product_count).to_frame()
    df_product_count = df_product_count.reset_index()
    df_product_count.rename(columns={'total_product':'product_count'}, inplace=True)
    df_cogs_product_df = pd.merge(df_product_sum, df_product_count)
    
    # prepare summary
    df_summary = pd.merge(df_revenue, df_cogs_product_df, how='left')
    df_summary = pd.merge(df_summary, df_cogs_id, how='left')
    df_summary['product_sum'] = df_summary['product_sum'].fillna(0)
    df_summary['product_count'] = df_summary['product_count'].fillna(0)
    # total cogs = by product id cogs + by product name cogs
    df_summary['cogs'] = df_summary['total_incoming'] + df_summary['product_sum']
    df_summary = pd.merge(df_summary, df_cogs_count)
    df_summary = pd.merge(df_summary, df_total_count)
    # total count = by package id count + by product count
    df_summary['total_count_incoming'] = df_summary['count_incoming'] + df_summary['product_count']
    df_summary['margin_$'] = df_summary['revenue'] - df_summary['cogs']
    df_summary['margin_%'] = df_summary['margin_$'] / df_summary['revenue']
    df_summary['coverage'] = df_summary['total_count_incoming'] / df_summary['total_count']
    df_summary_simp = df_summary[['date', 'revenue', 'cogs', 'margin_$', 'margin_%', 'total_count_incoming','product_count','count_incoming', 'coverage']]
    
    return df_summary_simp,df_cogs_average_product




In [5]:
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'
#CURRENT_MONTH = '2022-02'

In [6]:
ca_df = pd.read_csv('cali_comp.csv',index_col = 0)
ca_df.columns = ['company_name','company_identifier','state','licenses']
ca_df['company_identifier'] = ca_df['company_identifier'].astype(str)

In [7]:
ca_df_clean = ca_df[~ca_df['company_identifier'].isin(['PL','WHT','GF','UR'])]
ca_df_clean.shape

(42, 4)

In [8]:
len(ca_df_clean['company_identifier'].unique())

42

In [9]:
ca_df_clean

,company_name,company_identifier,state,licenses
0,99 High Tide,99HT,CA,C10-0000279-LIC
4,Budee,BUD,CA,C9-0000464-LIC;C9-0000467-LIC;C9-0000444-LIC;C...
6,Cali Greens LLC,CG,CA,C9-0000157-LIC
7,California Patients Alliance,CPA,CA,C10-0000461-LIC;C9-0000348-LIC
8,California Patients Club,CPC,CA,C9-0000056-LIC
9,California Street Cannabis,CSC,CA,C10-0000918-LIC;C10-0000670-LIC
10,California's Choice Collective,CCC,CA,C12-0000087-LIC
11,Calm and Collective,CC,CA,C10-0000758-LIC
15,Dubs Green Garden,DGG,CA,C9-0000016-LIC
17,E-Leaf,EL,CA,C9-0000370-LIC


In [10]:
def calculate_ca_gmv_change(company_identifier,transfer_packages_start_date,sales_transactrions_start_date):
    # setup
    df_summary_simp_list = {}
    quality_metric_list = list()
    # read data
    df_in,df_sales_deduped,bad_dl,unknown_transfer,rwp,miss_incoming,miss_receipts = data_quality_checks.run(company_identifier,transfer_packages_start_date,sales_transactrions_start_date)    
    licenses = df_sales_deduped['license_number'].unique()
    # loop thru locations
    for l in tqdm(licenses):
        print(l)
        df_in_l = df_in[df_in['license_number'] == l]
        df_sales_deduped_l = df_sales_deduped[df_sales_deduped['license_number'] == l]
        df_in_l['per_unit_incoming'] = df_in_l['shipper_wholesale_price'] / df_in_l['shipped_quantity']
        df_in_l = df_in_l[df_in_l['per_unit_incoming'] <= 10000]
        df_sales_l = df_sales_deduped_l
        df_summary_simp,df_cogs_average_product = cogs_analysis(df_in_l,df_sales_l,'monthly')
        df_summary_simp.index = df_summary_simp.date 
        # tax treatment
        df_summary_simp['revenue_after_tax'] = df_summary_simp['revenue'] * 1.15
        df_summary_simp['cogs_after_tax'] = df_summary_simp['cogs'] * 1.27
        df_summary_simp['margin_$_after_tax'] = df_summary_simp['revenue_after_tax'] - df_summary_simp['cogs_after_tax']
        df_summary_simp['margin_%_after_tax'] = df_summary_simp['margin_$_after_tax'] / df_summary_simp['revenue_after_tax']
        
        #revenue change
        df_summary_simp['revenue_change'] = df_summary_simp['revenue'].pct_change()
        df_summary_simp['revenue_change_after_tax'] = df_summary_simp['revenue_after_tax'].pct_change()
        
        #rolling gm %
        df_summary_simp['gm_past_quarter_after_tax'] = df_summary_simp[['margin_%_after_tax']].rolling(3).mean().values
        df_summary_simp['gm_past_2quarters_after_tax'] = df_summary_simp[['margin_%_after_tax']].rolling(6).mean().values
        df_summary_simp['gm_past_3quarters_after_tax'] = df_summary_simp[['margin_%_after_tax']].rolling(9).mean().values
        
        #rolling gm $
        df_summary_simp['gm$_past_quarter_after_tax'] = df_summary_simp[['margin_$_after_tax']].rolling(3).mean().values
        df_summary_simp['gm$_past_2quarters_after_tax'] = df_summary_simp[['margin_$_after_tax']].rolling(6).mean().values
        df_summary_simp['gm$_past_3quarters_after_tax'] = df_summary_simp[['margin_$_after_tax']].rolling(9).mean().values
        df_summary_simp_list[l] = df_summary_simp
            
    quality_metric_list.append(list([bad_dl,unknown_transfer,rwp,miss_incoming,miss_receipts]))       
    return df_summary_simp_list,quality_metric_list
    

In [13]:
def get_ca_gmv_change_bm_list(company_list,transfer_packages_start_date,sales_transactrions_start_date):
    cogs_df_list = {}
    quality_metric_list = {}
    for c in tqdm(company_list):
        print(c)
        cogs_df,quality_metric = calculate_ca_gmv_change([c],transfer_packages_start_date,sales_transactrions_start_date)
        cogs_df_list[c] = cogs_df
        quality_metric_list[c] = quality_metric
    return cogs_df_list,quality_metric_list
    
    

In [ ]:
#test out
#a,b = get_ca_gmv_change_bm_list(list(ca_df_clean['company_identifier'])[0:2],TRANSFER_PACKAGES_START_DATE,SALES_TRANSACTIONS_START_DATE,CURRENT_MONTH)


# run

In [ ]:
# no PL, WHT
# no GF (bad download history)
#UR: Data too big???

In [14]:
ca_cogs_df_list,ca_quality_metric_list = get_ca_gmv_change_bm_list(list(ca_df_clean['company_identifier']),TRANSFER_PACKAGES_START_DATE,SALES_TRANSACTIONS_START_DATE)


  0%|          | 0/42 [00:00<?, ?it/s]

99HT
Verifying download summaries for license C10-0000279-LIC...
Earliest download summary: 2021-01-01
Latest download summary: 2022-03-15

[SUCCESS] All download summaries look good!
[SUCCESS] No unknown transfer packages!
93.0% of incoming transfer packages have receiver wholesale price
# transactions missing incoming transfer package: 8210 (8.031382062920645%)
# transactions total: 102224
# receipts with mismatching transactions: 0 (0.0%)
# receipts missing transactions: 0 (0.0%)
# receipts total: 47357



  0%|          | 0/1 [00:00<?, ?it/s]

C10-0000279-LIC



  2%|▏         | 1/42 [00:31<21:16, 31.13s/it]

...

C9-0000082-LIC



 98%|█████████▊| 41/42 [28:15<00:32, 32.20s/it]

C10-0000817-LIC
VOY
Verifying download summaries for license C10-0000802-LIC...
Earliest download summary: 2020-01-01
Latest download summary: 2022-03-16

Verifying download summaries for license C12-0000030-LIC...
Earliest download summary: 2020-01-01
Latest download summary: 2022-03-16

Verifying download summaries for license C12-0000159-LIC...
Earliest download summary: 2020-01-01
Latest download summary: 2022-03-16

Verifying download summaries for license C12-0000175-LIC...
Earliest download summary: 2020-01-01
Latest download summary: 2022-03-16

[SUCCESS] All download summaries look good!
[FAILURE] Fxound a total of 14019 unknown transfer packages
15.0% of incoming transfer packages have receiver wholesale price
# transactions missing incoming transfer package: 0 (0.0%)
# transactions total: 5005
# receipts with mismatching transactions: 0 (0.0%)
# receipts missing transactions: 0 (0.0%)
# receipts total: 2069



100%|██████████| 42/42 [28:38<00:00, 40.93s/it]

C10-0000802-LIC


In [15]:
ca_cogs_df_list.keys()

dict_keys(['99HT', 'BUD', 'CG', 'CPA', 'CPC', 'CSC', 'CCC', 'CC', 'DGG', 'EL', 'EMA', 'EMF', 'EMM', 'EMT', 'GT', 'GRG', 'GHC', 'HB', 'HPCC', 'HC', 'IDC', 'LBC', 'ML', 'MW', 'MPW', 'MD', 'ND', 'PRP', 'PRL', 'RA', 'RGD', 'SFV', 'SV', 'SLCC', 'SO', 'ST', 'TGL', 'TJR', 'TT', 'T4L', 'UHHC', 'VOY'])

# output data

In [16]:
df = pd.DataFrame()
for k in ca_cogs_df_list.keys():
    print(k)
    for l in ca_cogs_df_list[k].keys():
        print(l)
        data = ca_cogs_df_list[k][l]
        data['company'] = k
        data['location'] = l
        print(data.shape)
        df = df.append(data)
        #data.to_excel('ca_location_analysis/'+'analysis_' + k + '_'+ l+'.xlsx')

99HT
C10-0000279-LIC
(15, 23)
BUD
C9-0000464-LIC
(5, 23)
C9-0000467-LIC
(4, 23)
C9-0000399-LIC
(12, 23)
C9-0000444-LIC
(4, 23)
C9-0000167-LIC
(11, 23)
CG
C9-0000157-LIC
(25, 23)
CPA
C10-0000461-LIC
(26, 23)
C9-0000348-LIC
(16, 23)
CPC
C9-0000056-LIC
(19, 23)
CSC
C10-0000670-LIC
(27, 23)
C10-0000918-LIC
(4, 23)
CCC
C12-0000087-LIC
(27, 23)
CC
C10-0000758-LIC
(13, 23)
DGG
C9-0000016-LIC
(27, 23)
EL
C9-0000370-LIC
(19, 23)
EMA
C10-0000774-LIC
(15, 23)
EMF
C10-0000939-LIC
(2, 23)
EMM
C10-0000786-LIC
(13, 23)
EMT
C10-0000695-LIC
(22, 23)
GT
C9-0000385-LIC
(5, 23)
C9-0000170-LIC
(9, 23)
GRG
C10-0000169-LIC
(27, 23)
C10-0000408-LIC
(8, 23)
GHC
C10-0000414-LIC
(17, 23)
HB
C10-0000339-LIC
(27, 23)
C10-0000624-LIC
(27, 23)
C10-0000301-LIC
(20, 23)
HPCC
C10-0000064-LIC
(27, 23)
C10-0000005-LIC
(27, 23)
HC
C9-0000341-LIC
(13, 23)
IDC
C9-0000320-LIC
(24, 23)
LBC
C9-0000154-LIC
(23, 23)
ML
C9-0000300-LIC
(21, 23)
MW
C10-0000332-LIC
(27, 23)
MPW
C10-0000547-LIC
(21, 23)
MD
C9-0000247-LIC
(27, 23)
ND


In [48]:
df

,date,revenue,cogs,margin_$,margin_%,total_count_incoming,product_count,count_incoming,coverage,revenue_after_tax,cogs_after_tax,margin_$_after_tax,margin_%_after_tax,revenue_change,revenue_change_after_tax,gm_past_quarter_after_tax,gm_past_2quarters_after_tax,gm_past_3quarters_after_tax,gm$_past_quarter_after_tax,gm$_past_2quarters_after_tax,gm$_past_3quarters_after_tax,company,location,id
0,2021-01,291646.25,69944.725609,221701.524391,0.760173,4808.0,2003.0,2805,0.696005,335393.1875,88829.801523,246563.385977,0.735147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99HT,C10-0000279-LIC,"(99HT, C10-0000279-LIC)"
1,2021-02,262906.73,85322.341897,177584.388103,0.675465,5464.0,377.0,5087,0.948611,302342.7395,108359.374209,193983.365291,0.641601,-0.098542,-0.098542,NaN,NaN,NaN,NaN,NaN,NaN,99HT,C10-0000279-LIC,"(99HT, C10-0000279-LIC)"
2,2021-03,315154.72,99950.244014,215204.475986,0.682853,6902.0,159.0,6743,0.951475,362427.9280,126936.809898,235491.118102,0.649760,0.198732,0.198732,0.675503,NaN,NaN,225345.956457,NaN,NaN,99HT,C10-0000279-LIC,"(99HT, C10-0000279-LIC)"
3,2021-04,329925.48,100937.302787,228988.177213,0.694060,7081.0,112.0,6969,0.931465,379414.3020,128190.374539,251223.927461,0.662136,0.046868,0.046868,0.651166,NaN,NaN,226899.470285,NaN,NaN,99HT,C10-0000279-LIC,"(99HT, C10-0000279-LIC)"
4,2021-05,336793.80,99047.578798,237746.221202,0.705910,7286.0,131.0,7155,0.918326,387312.8700,125790.425074,261522.444926,0.675223,0.020818,0.020818,0.662373,NaN,NaN,249412.496830,NaN,NaN,99HT,C10-0000279-LIC,"(99HT, C10-0000279-LIC)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,2021-11,18761.21,5470.325953,13290.884047,0.708424,610.0,52.0,558,0.884058,21575.3915,6947.313960,14628.077540,0.677998,0.111837,0.111837,0.591387,NaN,NaN,12210.382620,NaN,NaN,VOY,C10-0000802-LIC,"(VOY, C10-0000802-LIC)"
995,2021-12,32715.38,12198.903894,20516.476106,0.627120,1062.0,129.0,933,0.919481,37622.6870,15492.607945,22130.079055,0.588211,0.743778,0.743778,0.644411,NaN,NaN,16567.283526,NaN,NaN,VOY,C10-0000802-LIC,"(VOY, C10-0000802-LIC)"
996,2022-01,24339.01,11123.403847,13215.606153,0.542980,831.0,43.0,788,0.936866,27989.8615,14126.722885,13863.138615,0.495291,-0.256038,-0.256038,0.587167,NaN,NaN,16873.765070,NaN,NaN,VOY,C10-0000802-LIC,"(VOY, C10-0000802-LIC)"
997,2022-02,24450.82,10410.082937,14040.737063,0.574244,796.0,37.0,759,0.953293,28118.4430,13220.805331,14897.637669,0.529817,0.004594,0.004594,0.537773,0.564580,NaN,16963.618446,14587.000533,NaN,VOY,C10-0000802-LIC,"(VOY, C10-0000802-LIC)"


In [ ]:
#df.to_csv('ca_analysis_0307.csv')

In [ ]:
#df.to_excel('ca_location_analysis/'+'ca_analysis.xlsx')

In [17]:
df.head()

,date,revenue,cogs,margin_$,margin_%,total_count_incoming,product_count,count_incoming,coverage,revenue_after_tax,cogs_after_tax,margin_$_after_tax,margin_%_after_tax,revenue_change,revenue_change_after_tax,gm_past_quarter_after_tax,gm_past_2quarters_after_tax,gm_past_3quarters_after_tax,gm$_past_quarter_after_tax,gm$_past_2quarters_after_tax,gm$_past_3quarters_after_tax,company,location
date,,,,,,,,,,,,,,,,,,,,,,,
2021-01,2021-01,291646.25,69944.725609,221701.524391,0.760173,4808.0,2003.0,2805,0.696005,335393.1875,88829.801523,246563.385977,0.735147,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99HT,C10-0000279-LIC
2021-02,2021-02,262906.73,85322.341897,177584.388103,0.675465,5464.0,377.0,5087,0.948611,302342.7395,108359.374209,193983.365291,0.641601,-0.098542,-0.098542,NaN,NaN,NaN,NaN,NaN,NaN,99HT,C10-0000279-LIC
2021-03,2021-03,315154.72,99950.244014,215204.475986,0.682853,6902.0,159.0,6743,0.951475,362427.9280,126936.809898,235491.118102,0.649760,0.198732,0.198732,0.675503,NaN,NaN,225345.956457,NaN,NaN,99HT,C10-0000279-LIC
2021-04,2021-04,329925.48,100937.302787,228988.177213,0.694060,7081.0,112.0,6969,0.931465,379414.3020,128190.374539,251223.927461,0.662136,0.046868,0.046868,0.651166,NaN,NaN,226899.470285,NaN,NaN,99HT,C10-0000279-LIC
2021-05,2021-05,336793.80,99047.578798,237746.221202,0.705910,7286.0,131.0,7155,0.918326,387312.8700,125790.425074,261522.444926,0.675223,0.020818,0.020818,0.662373,NaN,NaN,249412.496830,NaN,NaN,99HT,C10-0000279-LIC


In [18]:
df['id'] = list(zip(df.company, df.location))
df = df.reset_index(drop = True)

In [53]:
df[df['company'] == 'EL']['margin_%_after_tax']

235    0.518352
236    0.546808
237    0.556164
238    0.559428
239    0.597249
240    0.595831
241    0.609921
242    0.615315
243    0.605307
244    0.610099
245    0.621055
246    0.622735
247    0.622396
248    0.607659
249    0.617833
250    0.632256
251    0.628641
252    0.614842
253    0.626042
Name: margin_%_after_tax, dtype: float64

In [21]:
df_high_coverage = df[df['coverage'] >= 0.7]

In [22]:
df_high_coverage.shape

(927, 24)

In [23]:
len(df_high_coverage['location'].unique())

57

In [24]:
df_high_coverage[['date','id']].groupby(['date']).count()

,id
date,
2020-01,6
2020-02,14
2020-03,18
2020-04,19
2020-05,21
2020-06,23
2020-07,25
2020-08,27
2020-09,26


# 3m

In [ ]:
df_high_coverage_3m_trim = df_high_coverage[(df_high_coverage['gm_past_quarter_after_tax'] > 0.2) & (df_high_coverage['gm_past_quarter_after_tax'] < 0.8)]



In [ ]:
#df_high_coverage_3m_trim.shape[0] / df_high_coverage.shape[0]

In [ ]:
df_high_coverage.head()

In [ ]:
orig_gm_past_3m = df_high_coverage_3m_trim[df_high_coverage_3m_trim['gm_past_quarter_after_tax'] > 0][['id','date']].groupby(['date']).count()
orig_gm_dolloar = df_high_coverage_3m_trim[df_high_coverage_3m_trim['gm_past_quarter_after_tax'] > 0][['margin_$_after_tax','date']].groupby(['date']).mean()
for i in numpy.arange(0.4, 0.61, 0.01):
    print(round(i,2))
    qualify = df_high_coverage_3m_trim[df_high_coverage_3m_trim['gm_past_quarter_after_tax'] > i]
    to_join = qualify[['id','date']].groupby(['date']).count()
    to_join_med_margin_dolloar = qualify[['margin_$_after_tax','date']].groupby(['date']).mean()
    to_join.columns = [str(round(i,2))]
    to_join_med_margin_dolloar.columns = [str(round(i,2))]
    orig_gm_past_3m = pd.concat([orig_gm_past_3m,to_join],axis = 1,join = 'outer')
    orig_gm_dolloar = pd.concat([orig_gm_dolloar,to_join_med_margin_dolloar],axis = 1,join = 'outer')

In [ ]:
orig_gm_past_3m

In [ ]:
df_high_coverage_3m_trim

In [ ]:
df_high_coverage_3m_trim[['date','margin_%_after_tax']].groupby(['date']).quantile(.9)

In [ ]:
df_high_coverage_3m_trim[(df_high_coverage_3m_trim['revenue'] >= 3.307699e+05)][['date','gm_past_quarter_after_tax']].groupby(['date']).quantile(.75)



In [ ]:
df_high_coverage_3m_trim[['date','gm_past_quarter_after_tax']].groupby(['date']).quantile(.75).plot(marker = 'o')

In [ ]:
# orig_gm_dolloar.mean(axis = 0).plot(marker = 'o',label = 'mean')
# orig_gm_dolloar.median(axis = 0).plot(marker = 'o',label = 'median')
# plt.legend()

In [ ]:
# orig_gm_dolloar['0.48'].plot()
# orig_gm_dolloar['0.55'].plot()
# plt.legend()

# 6m

In [ ]:
df_high_coverage_6m_trim = df_high_coverage[(df_high_coverage['gm_past_2quarters_after_tax'] > 0.2) & (df_high_coverage['gm_past_2quarters_after_tax'] < 0.8)]



In [ ]:
df_high_coverage_6m_trim[['date','gm_past_2quarters_after_tax','gm_past_quarter_after_tax','gm_past_3quarters_after_tax']].groupby(['date']).quantile(.25).plot(marker = 'o')

In [ ]:
orig_gm_past_6m = df_high_coverage_6m_trim[df_high_coverage_6m_trim['gm_past_2quarters_after_tax'] > 0][['id','date']].groupby(['date']).count()
for i in numpy.arange(0.4, 0.61, 0.01):
    print(round(i,2))
    qualify = df_high_coverage_6m_trim[df_high_coverage_6m_trim['gm_past_2quarters_after_tax'] > i]
    to_join = qualify[['id','date']].groupby(['date']).count()
    to_join.columns = [str(round(i,2))]
    orig_gm_past_6m = pd.concat([orig_gm_past_6m,to_join],axis = 1,join = 'outer')

In [ ]:
orig_gm_past_6m

In [ ]:
df_high_coverage_6m_trim[['date','gm_past_2quarters_after_tax']].groupby(['date']).quantile(1)

# 9m

In [ ]:
df_high_coverage_9m_trim = df_high_coverage[(df_high_coverage['gm_past_3quarters_after_tax'] > 0.2) & (df_high_coverage['gm_past_3quarters_after_tax'] < 0.8)]



In [ ]:
orig_gm_past_9m = df_high_coverage_9m_trim[df_high_coverage_9m_trim['gm_past_3quarters_after_tax'] > 0][['id','date']].groupby(['date']).count()
for i in numpy.arange(0.4, 0.61, 0.01):
    print(round(i,2))
    qualify = df_high_coverage_9m_trim[df_high_coverage_9m_trim['gm_past_3quarters_after_tax'] > i]
    to_join = qualify[['id','date']].groupby(['date']).count()
    to_join.columns = [str(round(i,2))]
    orig_gm_past_9m = pd.concat([orig_gm_past_9m,to_join],axis = 1,join = 'outer')

In [ ]:
orig_gm_past_9m

In [ ]:
df_high_coverage_9m_trim[['date','gm_past_3quarters_after_tax']].groupby(['date']).quantile(.75)

# percentile

In [ ]:
def calculate_percentile(n_list):
    return [stats.percentileofscore(n_list.dropna(),i, kind='weak') for i in n_list.dropna()]

In [ ]:
df[df['date'] =='2021-01']['gm_past_quarter_after_tax'].dropna()

In [ ]:
calculate_percentile(df[df['date'] =='2021-01']['gm_past_quarter_after_tax'])

In [ ]:
from scipy import stats
stats.percentileofscore(df['gm_past_quarter_after_tax'].dropna(),0 , kind='weak')

In [ ]:
## flowhub

In [ ]:
fh = pd.read_csv('flowhub_batch2_location_analysis_0222.csv',index_col= 0)

In [ ]:
fh[fh['location_state'] == 'CA']['margin_perc'].describe()

# flowhub

In [ ]:
fh = pd.read_csv('margin_distr_trim_after_tax_0310.csv',index_col= 0)

In [ ]:
fh

In [ ]:
final_df_trim = pd.read_csv('final_df_trim.csv',index_col= 0)

In [ ]:
final_df_trim

In [ ]:
orig_gm_past_3m_fh = final_df_trim[final_df_trim['gm_past_quarter_after_tax'] > 0][['id','date']].groupby(['date']).count()

for i in numpy.arange(0.4, 0.61, 0.01):
    print(round(i,2))
    qualify = df_high_coverage_3m_trim[df_high_coverage_3m_trim['gm_past_quarter_after_tax'] > i]
    to_join = qualify[['id','date']].groupby(['date']).count()
    to_join_med_margin_dolloar = qualify[['margin_$_after_tax','date']].groupby(['date']).mean()
    to_join.columns = [str(round(i,2))]
    to_join_med_margin_dolloar.columns = [str(round(i,2))]
    orig_gm_past_3m = pd.concat([orig_gm_past_3m,to_join],axis = 1,join = 'outer')
    orig_gm_dolloar = pd.concat([orig_gm_dolloar,to_join_med_margin_dolloar],axis = 1,join = 'outer')

# GM$

In [25]:
import seaborn as sns

## 3M

In [32]:
df_high_coverage_gm_trim_3m = df_high_coverage[(df_high_coverage['gm$_past_quarter_after_tax'] > 0)&(df_high_coverage['gm$_past_quarter_after_tax'] < 500000)]




In [36]:
df_high_coverage_gm_trim_3m[['date','gm$_past_quarter_after_tax']].groupby(['date']).quantile(.75)


,gm$_past_quarter_after_tax
date,
2020-03,215893.642407
2020-04,233065.077803
2020-05,209355.366723
2020-06,229920.961377
2020-07,220435.098592
2020-08,227856.286478
2020-09,174086.241952
2020-10,120896.536711
2020-11,118076.100219


## 6m

In [38]:
df_high_coverage_gm_trim_6m = df_high_coverage[(df_high_coverage['gm$_past_2quarters_after_tax'] > 0)&(df_high_coverage['gm$_past_2quarters_after_tax'] < 500000)]




In [42]:
df_high_coverage_gm_trim_6m[['date','gm$_past_quarter_after_tax']].groupby(['date']).quantile(.75)

,gm$_past_quarter_after_tax
date,
2020-06,229940.426296
2020-07,247137.778695
2020-08,227856.286478
2020-09,199780.410125
2020-10,120896.536711
2020-11,165047.446913
2020-12,114445.509543
2021-01,116514.198801
2021-02,195026.187621


## 9M

In [43]:
df_high_coverage_gm_trim_9m = df_high_coverage[(df_high_coverage['gm$_past_3quarters_after_tax'] > 0)&(df_high_coverage['gm$_past_3quarters_after_tax'] < 500000)]




In [46]:
df_high_coverage_gm_trim_9m[['date','gm$_past_quarter_after_tax']].groupby(['date']).quantile(.75)

,gm$_past_quarter_after_tax
date,
2020-09,225474.578299
2020-10,276507.668026
2020-11,185916.829432
2020-12,115591.134304
2021-01,116514.198801
2021-02,201067.776234
2021-03,201954.975387
2021-04,186563.097304
2021-05,186032.305333


In [ ]:
df_high_coverage_gm_trim = df_high_coverage[(df_high_coverage['margin_$_after_tax'] > 0)&(df_high_coverage['margin_$_after_tax'] < 500000)]

In [ ]:
df_high_coverage['margin_$_after_tax'].quantile(0.9)

In [ ]:
df_high_coverage[df_high_coverage['margin_$_after_tax'] > 800000]

In [ ]:
df_high_coverage_gm_trim.shape[0] / df_high_coverage.shape[0]

In [ ]:
sns.boxplot(df_high_coverage['margin_$_after_tax'])

In [ ]:
df_high_coverage['margin_$_after_tax'].describe()

In [ ]:
df_high_coverage['margin_$_after_tax'].quantile(0.9)

In [ ]:
stats.percentileofscore(df_high_coverage['margin_$_after_tax'],270000, kind='strict')

In [ ]:
plt.hist(df_high_coverage['margin_$_after_tax'],bins = 100)

In [ ]:
df_high_coverage_gm_trim[['date','margin_$_after_tax']].groupby(['date']).quantile(.9)


In [ ]:
df_high_coverage[['date','margin_$_after_tax']].groupby(['date']).quantile(.5)